# Estimación de abundancia y densidad mediante métodos de captura-marcado-recaptura

población cerrada

$\hat{p}$ varía en el tiempo (e.g. meterología, esfuerzo de captura), el espacio (e.g. hábitat), entre los individuos (sexo, edad, experiencia previa)

1. Población cerrada.

2. P(recaptura) = p(captura) y la distribución de probabilidad es homogenea

3. No existe perdida de marcas.

a. Atracción por la trampa y/o cebo.

> aumenta la P(captura)

b. Acostumbramiento a la trampa.

> aumenta P(captura | marcado)

c. Diferencias en técnicas de muestreo: tamaño de trampa, tipo de trampa

> Altera la P(captura)

d. Pérdida de las marcas.

> P(perdida marca)

e. Efecto de la marca en la sobrevivencia de los individuos

> P(muerte | marcado)

f. Efecto de la cacería en las estimaciones de tamaño poblacional

> P(muerte) > 0

## Supuesto genérales

a. Todos los individuos tienen igual e independiente probabilidad de ser capturados: muestras al azar.

> Interpretación: las capturas son estádisticamente independientes e identicamente distribuidas (i.e. es una muestra aleatoria) con una distribución de probabilidad uniforme.

$$\forall x_i \in \{\text{muestra}\}, \; P\{x_i = captura\} = P\{X = captura\}$$
$$X \sim U(0,1)$$

b. No cambia la proporción marcados:no marcados, por nacimientos o inmigraciones.

> Interpretación: es una población cerrada a entradas.

c. Las pérdidas de individuos por muerte o emigración afecta igual a marcados y no marcados.

> Interpretación: la población es abierta a salidas, ocurren por muerte o emigración y son estádisticamente independientes de la marca.

$$P\{\text{muerte} \lor \text{emigrar} | \text{marcado}\} = P\{\text{muerte} \lor \text{emigrar} \}$$

d. Los individuos marcados se "distribuyen homogeneamente" con respecto a los no marcados.

> Interpretación: no me es claro que quiere decir.

e. Las marcas no afectan la probabilidad de recaptura.

> Interpretación: la captura/recaptura es estádisticamente independiente de la presencia de marca.

$$P\{\text{captura} | \text{marca}\} = P\{\text{captura} | \text{NO marca}\} = P\{\text{captura}\}$$